In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
%cd drive/My Drive/IASD/DHAI

/content/drive/My Drive/IASD/DHAI


In [ ]:
!pip3 install datasets transformers huggingface_hub

In [ ]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
from datasets import load_dataset
imdb = load_dataset("imdb")

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = imdb["train"].map(preprocess_function, batched=True)
tokenized_test = imdb["test"].map(preprocess_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-56f49e4bcdf924ad.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-31cd5cf05201bc88.arrow


In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

In [ ]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [23]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [24]:
!git config --global credential.helper store

In [ ]:
from transformers import TrainingArguments, Trainer
 
repo_name = "finetuning-sentiment-model-25000-samples"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=5,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/gui-marra/finetuning-sentiment-model-25000-samples into local empty directory.


In [15]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7815


Step,Training Loss
500,0.315500
1000,0.251500
1500,0.239400
2000,0.170700


Saving model checkpoint to finetuning-sentiment-model-25000-samples/checkpoint-1563
Configuration saved in finetuning-sentiment-model-25000-samples/checkpoint-1563/config.json
Model weights saved in finetuning-sentiment-model-25000-samples/checkpoint-1563/pytorch_model.bin
tokenizer config file saved in finetuning-sentiment-model-25000-samples/checkpoint-1563/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-25000-samples/checkpoint-1563/special_tokens_map.json
tokenizer config file saved in finetuning-sentiment-model-25000-samples/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-25000-samples/special_tokens_map.json


Step,Training Loss
500,0.315500
1000,0.251500
1500,0.239400
2000,0.170700
2500,0.155700
3000,0.166100
3500,0.101600
4000,0.100500
4500,0.089400
5000,0.071900


Saving model checkpoint to finetuning-sentiment-model-25000-samples/checkpoint-3126
Configuration saved in finetuning-sentiment-model-25000-samples/checkpoint-3126/config.json
Model weights saved in finetuning-sentiment-model-25000-samples/checkpoint-3126/pytorch_model.bin
tokenizer config file saved in finetuning-sentiment-model-25000-samples/checkpoint-3126/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-25000-samples/checkpoint-3126/special_tokens_map.json
Saving model checkpoint to finetuning-sentiment-model-25000-samples/checkpoint-4689
Configuration saved in finetuning-sentiment-model-25000-samples/checkpoint-4689/config.json
Model weights saved in finetuning-sentiment-model-25000-samples/checkpoint-4689/pytorch_model.bin
tokenizer config file saved in finetuning-sentiment-model-25000-samples/checkpoint-4689/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-25000-samples/checkpoint-4689/special_tokens_map.json
Saving model

TrainOutput(global_step=7815, training_loss=0.12016724984918889, metrics={'train_runtime': 5803.7588, 'train_samples_per_second': 21.538, 'train_steps_per_second': 1.347, 'total_flos': 1.6390845293487744e+16, 'train_loss': 0.12016724984918889, 'epoch': 5.0})

In [16]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16


{'epoch': 5.0,
 'eval_accuracy': 0.9314,
 'eval_f1': 0.932017283069727,
 'eval_loss': 0.3710886240005493,
 'eval_runtime': 411.1376,
 'eval_samples_per_second': 60.807,
 'eval_steps_per_second': 3.802}

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to finetuning-sentiment-model-25000-samples
Configuration saved in finetuning-sentiment-model-25000-samples/config.json
Model weights saved in finetuning-sentiment-model-25000-samples/pytorch_model.bin
tokenizer config file saved in finetuning-sentiment-model-25000-samples/tokenizer_config.json
Special tokens file saved in finetuning-sentiment-model-25000-samples/special_tokens_map.json
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


In [ ]:
from transformers import pipeline
 
sentiment_model = pipeline(model="gui-marra/finetuning-sentiment-model-3000-samples")
sentiment_model(["I love this move", "This movie sucks!"])